# Simple NNs with custom embeddings

## Importing data

In [7]:
import copy
from itertools import product
from numpy import isnan
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext.data as data
import sys
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../data_pipeline/')
import preprocessing as pre
from training import TrainingModule

SEED = 1312
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
train_data, test_data, val_data, TEXT, LABEL = pre.get_data(
    'train_small.csv', 'val_small.csv', 'test_small.csv')

Connected!


In [3]:
USE_CUDA = torch.cuda.is_available()

TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

BATCH_SIZE = 5

train_it, test_it, val_it = data.BucketIterator.splits(
    (train_data, test_data, val_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.alj_text),
    sort_within_batch=True,
    device = torch.device('cuda' if USE_CUDA else 'cpu'))

## Simple NN Model

In [4]:
class WordEmbAvg(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx, two_layers=True, dropout_p=0.0):
        
        super().__init__()
        
        # Define an embedding layer, a couple of linear layers, and 
        # the ReLU non-linearity.

        ##YOUR CODE HERE##
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        if two_layers == True:
            self.linear1 = nn.Linear(embedding_dim, hidden_dim)
            self.linear2 = nn.Linear(hidden_dim, output_dim) 
        else:
            self.linear1 = nn.Linear(embedding_dim, output_dim)
            self.linear2 = None
        self.relu = nn.ReLU()
        self.drop_layer = nn.Dropout(p=dropout_p)

        
        
    def forward(self, text):

        ##YOUR CODE HERE##
        embedded = self.embedding(text)
        embedded = embedded.mean(0)
        if not self.linear2:
            linear1_output = self.linear1(embedded)
            output = self.relu(linear1_output)
            output = self.drop_layer(output)
            return output
        else:
            linear1_output = self.linear1(embedded)
            linear2_input = self.relu(linear1_output)
            output = self.linear2(linear2_input)
            output = self.drop_layer(output)
            return output

## Training models

In [5]:
# Store training results

df = pd.DataFrame(columns=['architecture', 'embeddings',
                           'hidden', 'dropouts',
                           'learning_rate', 'epochs',
                           'dev_acc', 'dev_prec', 'dev_recall',
                           'metric'])

# Model architecture parameters
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_SIZES = [32, 64, 128, 256]
HIDDEN_SIZES = [10, 25, 40, 50]
OUTPUT_SIZE = 1
DROPOUTS = [0, 0.1, 0.25, 0.5, 0.75]
PADDING_IDX = TEXT.vocab.stoi[TEXT.pad_token]

# Model training hyperparameters
LEARNING_RATE = [0.01, 0.001, 0.0001]
train_len = 0
train_pos = 0
for batch in train_it:
    train_len += len(batch.decision_binary)
    train_pos += batch.decision_binary.sum().item()
POS_WEIGHT = torch.tensor([(train_len - train_pos) / train_pos])
if USE_CUDA:
    POS_WEIGHT = POS_WEIGHT.cuda()
EPOCHS = 10

# Iterator over various model parameters
param_iter = product (EMBEDDING_SIZES, HIDDEN_SIZES,
                      DROPOUTS, LEARNING_RATE)

# Magic loop
best_acc = (None, None)
best_rec = (None, None)
best_prec = (None, None)
for i, (embed_size, hidden_size, dropout, lr) in enumerate(param_iter):
    print(f'Architecture #{i}\n' + '-' * 20)
    model = WordEmbAvg(INPUT_DIM, embed_size, hidden_size,
                OUTPUT_SIZE, PADDING_IDX, dropout_p=dropout)
    
    tm = TrainingModule(model, lr, POS_WEIGHT, USE_CUDA, EPOCHS)
    
    best_models = tm.train_model(train_it, val_it)
    
    for metric, best_model in best_models.items():
        row = [i, embed_size, hidden_size, dropout,
               lr, EPOCHS, best_model.accuracy,
               best_model.precision, best_model.recall, metric]
        df.loc[len(df)] = row
        if best_acc[0] is None or best_model.accuracy > best_acc[1]:
            best_acc = (copy.deepcopy(best_model.model), best_model.accuracy)
        if best_rec[0] is None or best_model.recall > best_rec[1]:
            best_rec = (copy.deepcopy(best_model.model), best_model.recall)
        if best_prec[0] is None or isnan(best_prec[1]) or\
          best_model.precision > best_prec[1]:
            best_prec = (copy.deepcopy(best_model.model), best_model.precision)
    
    print('-' * 20 + '\n')


Architecture #0
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8839
Epoch 1: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8762
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8557
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8205
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.7942
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.7450
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6790
Epoch 7: Dev Accuracy: 1.0000; Dev Precision: 1.0000; Dev Recall: 1.0000; Dev Loss:0.6361
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6018
Epoch 9: Dev Accuracy: 1.0000; Dev Precision: 1.0000; Dev Recall: 1.0000; Dev Loss:0.5483
--------------------

Architecture #1
--------------------
Epoch 0: Dev Accuracy: 0.750

Epoch 9: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8820
--------------------

Architecture #9
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.8955
Epoch 1: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8817
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8733
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8753
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8611
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8530
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8349
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8281
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8288
Epoch 9: Dev Accuracy: 0.5000; Dev Precision: 0.00

Epoch 6: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.8927
Epoch 7: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.8925
Epoch 8: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.8924
Epoch 9: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.8923
--------------------

Architecture #18
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.9093
Epoch 1: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8750
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8668
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8473
Epoch 4: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8087
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7449
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: 0.5000

Epoch 5: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.9002
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8997
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8995
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8995
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8994
--------------------

Architecture #27
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.9021
Epoch 1: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.9305
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.9156
Epoch 3: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8872
Epoch 4: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8860
Epoch 5: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Rec

Epoch 3: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9082
Epoch 4: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9081
Epoch 5: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9080
Epoch 6: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9080
Epoch 7: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9081
Epoch 8: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9081
Epoch 9: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9080
--------------------

Architecture #36
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8894
Epoch 1: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8820
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8787
Epoch 3: Dev Accuracy: 0.7500; Dev Precisio

...

Epoch 3: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7723
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8040
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.7374
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.6707
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.6460
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.5997
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6460
--------------------

Architecture #205
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.2500; Dev Recall: 1.0000; Dev Loss:0.9020
Epoch 1: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.9008
Epoch 2: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.9118
Epoch 3: Dev Accuracy: 0.5000; Dev Precision:

Epoch 0: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8540
Epoch 1: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:1.1552
Epoch 2: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.6981
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.6315
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.6641
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.5011
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6110
Epoch 7: Dev Accuracy: 1.0000; Dev Precision: 1.0000; Dev Recall: 1.0000; Dev Loss:0.3934
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.7203
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.7404
--------------------

Architecture #214
--------------------
Epoch 0: Dev Accuracy: 0.2500; Dev Precision: 0.

Epoch 8: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8745
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8746
--------------------

Architecture #222
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:1.1760
Epoch 1: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8796
Epoch 2: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.9284
Epoch 3: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8731
Epoch 4: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8946
Epoch 5: Dev Accuracy: 0.5000; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8879
Epoch 6: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8266
Epoch 7: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.7971
Epoch 8: Dev Accuracy: 0.7500; Dev Precisi

Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8854
Epoch 6: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8851
Epoch 7: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8846
Epoch 8: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8846
Epoch 9: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8840
--------------------

Architecture #231
--------------------
Epoch 0: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.8794
Epoch 1: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8522
Epoch 2: Dev Accuracy: 0.7500; Dev Precision: 0.5000; Dev Recall: 1.0000; Dev Loss:0.8712
Epoch 3: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.7120
Epoch 4: Dev Accuracy: 0.7500; Dev Precision: nan; Dev Recall: 0.0000; Dev Loss:0.6876
Epoch 5: Dev Accuracy: 0.7500; Dev Precision: 0.

Epoch 2: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8955
Epoch 3: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8954
Epoch 4: Dev Accuracy: 0.5000; Dev Precision: 0.3333; Dev Recall: 1.0000; Dev Loss:0.8951
Epoch 5: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8960
Epoch 6: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8965
Epoch 7: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8961
Epoch 8: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8962
Epoch 9: Dev Accuracy: 0.2500; Dev Precision: 0.0000; Dev Recall: 0.0000; Dev Loss:0.8968
--------------------



## Save results of model training

In [6]:
SAVE_PREFIX = '../results/SimpleNNCustom_'
df.to_csv(f'{SAVE_PREFIX}models.csv')
torch.save(best_acc[0], f'{SAVE_PREFIX}best_acc.pt')
torch.save(best_rec[0], f'{SAVE_PREFIX}best_rec.pt')
torch.save(best_prec[0], f'{SAVE_PREFIX}best_prec.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type WordEmbAvg. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
